In [1]:
import datetime

config = {
    "start_date": datetime.date.fromisoformat('2025-01-31'),
    "race_date": datetime.date.fromisoformat('2025-06-28'),
    "race_details": {
        "swim_distance": 1.5, # in km
        "bike_distance": 40, # in km
        "run_distance": 10 # in km
    },
    "current_fitness": 50, # 0-100
    "max_weekly_training": 10, # hours per week
    "max_block_weeks": 4 # maximum weeks per block
}

print(config)

{'start_date': datetime.date(2025, 1, 31), 'race_date': datetime.date(2025, 6, 28), 'race_details': {'swim_distance': 1.5, 'bike_distance': 40, 'run_distance': 10}, 'current_fitness': 50, 'max_weekly_training': 10, 'max_block_weeks': 4}


In [2]:
import pyomo.environ as pyo

def create_linear_periodization_plan(config):
    #TODO: validate incoming config (start date should be before race date, fitness should be in range, etc)

    
    available_weeks = []
    date_counter = config["start_date"] - datetime.timedelta(days=config["start_date"].weekday()) # weeks always start on monday
    while date_counter < config["race_date"]:
        next_week = date_counter + datetime.timedelta(days=7)
        available_weeks.append({ "start_date": date_counter, "end_date": next_week }) # note: next week is the first day of the next week
        date_counter = next_week

    weeks_periodized = assign_strategy_to_weeks(available_weeks,config)
    return weeks_periodized

def assign_strategy_to_weeks(available_weeks, config):
    #TODO: validate weeks incoming
    number_of_weeks = len(available_weeks)
    max_block_weeks = config["max_block_weeks"]

    print("number of weeks: ", number_of_weeks)

    available_week_types = {
        0: "rest",
        1: "peak",
        2: "base",
        3: "build"
    }

    #init pyomo model

    model = pyo.ConcreteModel()
    model.Weeks = pyo.RangeSet(0, number_of_weeks-1)
    
    model.AssignedValues = pyo.Var(model.Weeks, domain=pyo.NonNegativeIntegers, bounds=(0, 3))
    model.StartBlockWeeks = pyo.Var(model.Weeks, domain=pyo.Boolean)

    # maximize base periods and build periods
    model.obj = pyo.Objective(expr=sum(model.AssignedValues[n] for n in model.Weeks) - sum(model.StartBlockWeeks[n] for n in model.Weeks), sense=pyo.maximize)

    # constraints

    # last two weeks must not be assigned to base or build
    def peak_period_constraint_rule(m, week):
        if week > number_of_weeks - 3:
            return m.AssignedValues[week]  <= 1
        else:
            return pyo.Constraint.Skip
        
    model.peak_period_constraint = pyo.Constraint(model.Weeks, rule=peak_period_constraint_rule)

    # must have 12 base weeks before build

    def min_base_week_constraint_rule(m,week):   
        if week <= 12 :
            return m.AssignedValues[week] <= 2 
        else:
            return pyo.Constraint.Skip

    model.rest_week_constraint = pyo.Constraint(model.Weeks, rule=min_base_week_constraint_rule)

    # must start a block every time assigned[n] != assigned[n-1] - TODO: how to adjust this so it catches downward shifts as well?

    def start_block_on_transition_rule(m,week):
        if week == 0:
            return m.StartBlockWeeks[week] >= 1
        else:
            return m.AssignedValues[week] - m.AssignedValues[week - 1] - m.StartBlockWeeks[week] <= 0
            
    model.start_block_on_transition = pyo.Constraint(model.Weeks, rule=start_block_on_transition_rule)

    # must start a transition block at least every {max_block_weeks} weeks

    def start_block_periodically_rule(m,week):
        if week > max_block_weeks:
            week_window = range(week - max_block_weeks, week)
            return sum(m.StartBlockWeeks[n] for n in week_window) >= 1
        else:
            return pyo.Constraint.Skip

    model.start_block_periodically = pyo.Constraint(model.Weeks, rule=start_block_periodically_rule)
    
    # must not start a transition block more than {max_block_weeks - 1} weeks

    def start_block_periodically_rule_2(m,week):
        if week > max_block_weeks:
            week_window = range(week - (max_block_weeks - 1), week)
            return sum(m.StartBlockWeeks[n] for n in week_window) <= 1
        else:
            return pyo.Constraint.Skip
            

    model.start_block_periodically_2 = pyo.Constraint(model.Weeks, rule=start_block_periodically_rule_2)

    # solve
    
    solver = pyo.SolverFactory('glpk')
    solver.options['tmlim'] = 5
    results = solver.solve(model)
    

    # print results
    # print(results)
    model.pprint()


    # add strategy and if it is the start of a block to each week
    for idx, week in enumerate(available_weeks):
        week["strategy"] = available_week_types[model.AssignedValues[idx].value]
        week["start_block"] = model.StartBlockWeeks[idx].value == 1

    return available_weeks
    
    

week_plan = create_linear_periodization_plan(config)
print(week_plan)

for week in week_plan:
    print(week["start_date"], week["strategy"], week["start_block"])

        
                               

number of weeks:  22
1 RangeSet Declarations
    Weeks : Dimen=1, Size=22, Bounds=(0, 21)
        Key  : Finite : Members
        None :   True :  [0:21]

2 Var Declarations
    AssignedValues : Size=22, Index=Weeks
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          1 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          2 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          3 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          4 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          5 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          6 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          7 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          8 :     0 :   2.0 :     3 : False : False : NonNegativeIntegers
          9 :     0 :   2.0 :     3 : False : False : Non

In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.workout_planner import *



def print_to_csv(filename, week_plan):
    with open(filename, "wt") as file:
        writer = csv.writer(file)
        writer.writerow(["Plan for {} week starting {}".format(week_plan["strategy"],week_plan["start_date"])])
        writer.writerow(["date", "sport", "name", "intensity", "duration"])
        for day, day_workouts in enumerate(week_plan["workouts"]):
            for workout in day_workouts:
                writer.writerow([week_plan["start_date"] + datetime.timedelta(days=day), workout["sport"], workout["workout_name"], workout["intensity"], workout["duration"]])


                
for week_number, week in enumerate(week_plan):
    
    incoming_fatigue = { "bike": 30, "run": 30, "swim": 30 }
    incoming_fitness = { "bike": 50, "run": 50, "swim": 50 }

    if week_number > 0:
        incoming_fatigue = week_plan[week_number-1]["fatigue_outcome"]
        incoming_fitness = week_plan[week_number-1]["fitness_outcome"]


    is_last_week_of_block = False
    if week_number < len(week_plan) - 1:
        is_last_week_of_block = week_plan[week_number+1]["start_block"]
    
    week_with_workouts = add_workouts_to_week_plan(week, incoming_fatigue, incoming_fitness, is_last_week_of_block, 5)
    print(week_with_workouts)
    
    for idx, workout in enumerate(week_with_workouts["workouts"]):
        print(idx, workout)

    

    # write to a file
    print_to_csv("./test_workout_plan_week_{}.csv".format(week_number), week_with_workouts)


['swim', 'recovery swim', '1', '15', '1', '.2', '']
['swim', 'aerobic intervals', '4', '40', '10', '2', '']
['swim', 'tempo intervals', '6', '30', '20', '3', '']
['swim', 'muscular force reps', '7', '30', '30', '3', '']
['swim', 'open water current intervals', '8', '20', '30', '3', '']
['swim', 'aerobic intervals with paddles', '4', '30', '15', '2.5', '']
['swim', 'fast form 25s', '3', '20', '10', '1', '']
['swim', 'toy sets', '3', '20', '10', '1', '']
['swim', 'long cruise intervals', '8', '40', '60', '4', 'base']
['swim', 'short cruise intervals', '8', '30', '40', '3', 'base']
['swim', 'threshold', '7', '20', '50', '3', 'base']
['swim', 'vo2 max intervals', '9', '20', '70', '4', 'base']
['swim', 'anaerobic capacity intervals', '10', '20', '75', '4.5', 'base']
['bike', 'recovery bike', '1', '30', '1', '.2', '']
['bike', 'aerobic threshold', '3', '70', '30', '2', '']
['bike', 'intensive endurance', '7', '90', '40', '3', '']
['bike', 'force reps', '8', '40', '60', '3', '']
['bike', 'hil